In [ ]:
import os
import math
import json

from glob import glob

from IPython.display import HTML

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation


from PIL import Image

In [ ]:
!pip install git+https://github.com/ricardodeazambuja/AerialViewGenerator
from aerialviewgenerator.aerialview import AerialView
baseurl = 'https://wxs.ign.fr/choisirgeoportail/geoportail/wmts?REQUEST=GetTile&SERVICE=WMTS&VERSION=1.0.0&STYLE=normal&TILEMATRIXSET=PM&FORMAT=image/jpeg&LAYER=ORTHOIMAGERY.ORTHOPHOTOS&TILEMATRIX={z}&TILEROW={y}&TILECOL={x}'
FOV = 73.0
avg = AerialView(zoom=20, baseurl=baseurl)

In [ ]:
# altitude that landing is considered successful
# because we could finish landing from here using traditional depth sensing based landing
landed_altitude = 20

In [ ]:
s = 2
r = 0
rl = [0]
el = []
for i in range(100):
    r = r + (s - r)*0.1
    rl.append(r)
    el.append(s-s*np.exp(-i*0.1))

In [ ]:
plt.plot(rl)
plt.plot(el)

## Inspecting the results from DEFAULT

In [ ]:
# (sub)folder where the experiments were saved
main_folder = "DEFAULT"

experiment_folders = sorted(glob(main_folder + '/*/'))
results_default = {}
for folder in experiment_folders:
    experiment_name = folder.split('/')[1]
    main_filename = folder + experiment_name
    with open(main_filename + ".json", "r") as file:
        experiment_dict = json.load(file)
    with open(main_filename + ".txt", "r") as file:
        init_lat, init_lon = [float(v.split('=')[1]) for v in file.readline().split(',')]
    elapsed_times = list(experiment_dict.keys())

    positions = []
    success = False
    for ti in elapsed_times[1:]:
        positions.append(experiment_dict[ti]['position'][:2])
        if experiment_dict[ti]['position'][2] <= landed_altitude:
            success = True
            break
    positions = np.asarray(positions)
    diffPos = experiment_dict[ti]['position']
    bearing = math.degrees(math.atan2(-diffPos[1],diffPos[0]))+90
    distance = np.linalg.norm(diffPos[:2]) # relative distance
    final_lat, final_lon = avg.getPointAtDistance(init_lat, init_lon, distance, bearing)
    img = np.asarray(avg.getAerialImage(final_lat, final_lon, 0, diffPos[2], FOV, (324,324)))
    img100 = np.asarray(avg.getAerialImage(final_lat, final_lon, 0, 100, FOV, (324,324)))
    imginit = np.asarray(avg.getAerialImage(init_lat, init_lon, 0, 300, FOV, (324,324)))
    imginit100 = np.asarray(avg.getAerialImage(init_lat, init_lon, 0, 100, FOV, (324,324)))
    square_size = avg.squareAtCamera(300,FOV)
    square_size100 = avg.squareAtCamera(100,FOV)
    init_view_size = 324*square_size100/square_size
    pixel_per_meter = 324/square_size
    positions_pixel = positions*pixel_per_meter
    positions_pixel[:,0] += 324/2
    positions_pixel[:,1] = 324/2 - positions_pixel[:,1]
    fig, axs = plt.subplots(1,4, figsize=(15,10))
    fig.suptitle(f'{experiment_name}\nStart:{(init_lat, init_lon)}\nEnd:{(final_lat, final_lon)}', fontsize=16, y=0.8)
    axs[0].imshow(img)
    axs[0].set_title(f'Altitude {diffPos[2]:.1f}m')
    axs[0].axis('off')
    axs[1].imshow(img100)
    axs[1].set_title(f'Final position (alt. 100.0m)')
    axs[1].axis('off')
    axs[2].imshow(imginit)
    axs[2].plot([324/2],[324/2],marker='.',markersize=20,color='red')
    axs[2].plot(positions_pixel[:,0],positions_pixel[:,1],marker='.',markersize=3,color='red')
    axs[2].plot(positions_pixel[-1,0],positions_pixel[-1,1],marker='*',markersize=20,color='blue')
    rect = patches.Rectangle((324/2-init_view_size/2, 324/2-init_view_size/2),
                             init_view_size, init_view_size,
                             linewidth=2.5, linestyle='--', edgecolor='y', facecolor='none')
    axs[2].add_patch(rect)
    axs[2].set_title(f'Trajectory (alt. 300.0m, square size {square_size:.2f}m)')
    axs[2].axis('off')
    axs[3].imshow(imginit100)
    axs[3].plot([324/2],[324/2],marker='.',markersize=20,color='red')
    axs[3].set_title(f'Initial view (alt. 100.0m)')
    axs[3].axis('off')
    plt.tight_layout()
    plt.show()
    if success:
        keep_it = input("Is it any good? (y/n): ")
        if "y" in keep_it:
            results_default[experiment_name] = [experiment_dict,
                                                 (init_lat, init_lon),
                                                 (final_lat, final_lon)]

In [ ]:
with open("results_default.json", "w") as file:
    json.dump(results_default, file)

In [ ]:
len(results_default)

## Inspecting the results from PEACE

In [ ]:
# (sub)folder where the experiments were saved
main_folder = "PEACE"

experiment_folders = sorted(glob(main_folder + '/*/'))
results_peace = {}
for folder in experiment_folders:
    experiment_name = folder.split('/')[1]
    main_filename = folder + experiment_name
    with open(main_filename + ".json", "r") as file:
        experiment_dict = json.load(file)
    with open(main_filename + ".txt", "r") as file:
        init_lat, init_lon = [float(v.split('=')[1]) for v in file.readline().split(',')]
    elapsed_times = list(experiment_dict.keys())

    positions = []
    success = False
    for ti in elapsed_times[1:]:
        positions.append(experiment_dict[ti]['position'][:2])
        if experiment_dict[ti]['position'][2] <= landed_altitude:
            success = True
            break
    positions = np.asarray(positions)
    diffPos = experiment_dict[ti]['position']
    bearing = math.degrees(math.atan2(-diffPos[1],diffPos[0]))+90
    distance = np.linalg.norm(diffPos[:2]) # relative distance
    final_lat, final_lon = avg.getPointAtDistance(init_lat, init_lon, distance, bearing)
    img = np.asarray(avg.getAerialImage(final_lat, final_lon, 0, diffPos[2], FOV, (324,324)))
    img100 = np.asarray(avg.getAerialImage(final_lat, final_lon, 0, 100, FOV, (324,324)))
    imginit = np.asarray(avg.getAerialImage(init_lat, init_lon, 0, 300, FOV, (324,324)))
    imginit100 = np.asarray(avg.getAerialImage(init_lat, init_lon, 0, 100, FOV, (324,324)))
    square_size = avg.squareAtCamera(300,FOV)
    square_size100 = avg.squareAtCamera(100,FOV)
    init_view_size = 324*square_size100/square_size
    pixel_per_meter = 324/square_size
    positions_pixel = positions*pixel_per_meter
    positions_pixel[:,0] += 324/2
    positions_pixel[:,1] = 324/2 - positions_pixel[:,1]
    fig, axs = plt.subplots(1,4, figsize=(15,10))
    fig.suptitle(f'{experiment_name}\nStart:{(init_lat, init_lon)}\nEnd:{(final_lat, final_lon)}', fontsize=16, y=0.8)
    axs[0].imshow(img)
    axs[0].set_title(f'Altitude {diffPos[2]:.1f}m')
    axs[0].axis('off')
    axs[1].imshow(img100)
    axs[1].set_title(f'Final position (alt. 100.0m)')
    axs[1].axis('off')
    axs[2].imshow(imginit)
    axs[2].plot([324/2],[324/2],marker='.',markersize=20,color='red')
    axs[2].plot(positions_pixel[:,0],positions_pixel[:,1],marker='.',markersize=3,color='red')
    axs[2].plot(positions_pixel[-1,0],positions_pixel[-1,1],marker='*',markersize=20,color='blue')
    rect = patches.Rectangle((324/2-init_view_size/2, 324/2-init_view_size/2),
                             init_view_size, init_view_size,
                             linewidth=2.5, linestyle='--', edgecolor='y', facecolor='none')
    axs[2].add_patch(rect)
    axs[2].set_title(f'Trajectory (alt. 300.0m, square size {square_size:.2f}m)')
    axs[2].axis('off')
    axs[3].imshow(imginit100)
    axs[3].plot([324/2],[324/2],marker='.',markersize=20,color='red')
    axs[3].set_title(f'Initial view (alt. 100.0m)')
    axs[3].axis('off')
    plt.tight_layout()
    plt.show()
    if success:
        keep_it = input("Is it any good? (y/n): ")
        if "y" in keep_it:
            results_peace[experiment_name] = [experiment_dict,
                                                 (init_lat, init_lon),
                                                 (final_lat, final_lon)]

In [ ]:
with open("results_peace.json", "w") as file:
    json.dump(results_peace, file)

In [ ]:
len(results_peace)

## Inspecting the results from our system (original)

In [ ]:
# (sub)folder where the experiments were saved
main_folder = "Original"

experiment_folders = sorted(glob(main_folder + '/*/'))
results_original = {}
for folder in experiment_folders:
    experiment_name = folder.split('/')[1]
    main_filename = folder + experiment_name
    with open(main_filename + ".json", "r") as file:
        experiment_dict = json.load(file)
    with open(main_filename + ".txt", "r") as file:
        init_lat, init_lon = [float(v.split('=')[1]) for v in file.readline().split(',')]
    elapsed_times = list(experiment_dict.keys())

    positions = []
    for ti in elapsed_times[1:]:
        positions.append(experiment_dict[ti]['position'][:2])
        if experiment_dict[ti]['position'][2] <= landed_altitude:
            break
    positions = np.asarray(positions)
    diffPos = experiment_dict[ti]['position']
    bearing = math.degrees(math.atan2(-diffPos[1],diffPos[0]))+90
    distance = np.linalg.norm(diffPos[:2]) # relative distance
    final_lat, final_lon = avg.getPointAtDistance(init_lat, init_lon, distance, bearing)
    img = np.asarray(avg.getAerialImage(final_lat, final_lon, 0, diffPos[2], FOV, (324,324)))
    img100 = np.asarray(avg.getAerialImage(final_lat, final_lon, 0, 100, FOV, (324,324)))
    imginit = np.asarray(avg.getAerialImage(init_lat, init_lon, 0, 300, FOV, (324,324)))
    imginit100 = np.asarray(avg.getAerialImage(init_lat, init_lon, 0, 100, FOV, (324,324)))
    square_size = avg.squareAtCamera(300,FOV)
    square_size100 = avg.squareAtCamera(100,FOV)
    init_view_size = 324*square_size100/square_size
    pixel_per_meter = 324/square_size
    positions_pixel = positions*pixel_per_meter
    positions_pixel[:,0] += 324/2
    positions_pixel[:,1] = 324/2 - positions_pixel[:,1]
    fig, axs = plt.subplots(1,4, figsize=(15,10))
    fig.suptitle(f'{experiment_name}\nStart:{(init_lat, init_lon)}\nEnd:{(final_lat, final_lon)}', fontsize=16, y=0.8)
    axs[0].imshow(img)
    axs[0].set_title(f'Altitude {diffPos[2]:.1f}m')
    axs[0].axis('off')
    axs[1].imshow(img100)
    axs[1].set_title(f'Final position (alt. 100.0m)')
    axs[1].axis('off')
    axs[2].imshow(imginit)
    axs[2].plot([324/2],[324/2],marker='.',markersize=20,color='red')
    axs[2].plot(positions_pixel[:,0],positions_pixel[:,1],marker='.',markersize=3,color='red')
    axs[2].plot(positions_pixel[-1,0],positions_pixel[-1,1],marker='*',markersize=20,color='blue')
    rect = patches.Rectangle((324/2-init_view_size/2, 324/2-init_view_size/2),
                             init_view_size, init_view_size,
                             linewidth=2.5, linestyle='--', edgecolor='y', facecolor='none')
    axs[2].add_patch(rect)
    axs[2].set_title(f'Trajectory (alt. 300.0m, square size {square_size:.2f}m)')
    axs[2].axis('off')
    axs[3].imshow(imginit100)
    axs[3].plot([324/2],[324/2],marker='.',markersize=20,color='red')
    axs[3].set_title(f'Initial view (alt. 100.0m)')
    axs[3].axis('off')
    plt.tight_layout()
    plt.show()
    keep_it = input("Is it any good? (y/n): ")
    if "y" in keep_it:
        results_original[experiment_name] = [experiment_dict,
                                             (init_lat, init_lon),
                                             (final_lat, final_lon)]

In [ ]:
len(results_original)

In [ ]:
100*len(results_original)/50

In [ ]:
with open("results_original.json", "w") as file:
    json.dump(results_original, file)

## Comparing both results

In [ ]:
improvement_perc = 100*len(results_peace)/len(results_original)
print(f"Improvement from the use of PEACE (total success): {improvement_perc:.2f}%")

In [ ]:
possible_states = {'AIMING',
 'CLIMBING',
 'LANDING',
 'RESTARTING',
 'SEARCHING',
 'SENSOR_ERROR',
 'WAITING'}

In [ ]:
# total time until reaching landing altitude
# total travelled distance until reaching landing altitude
# horizontal distance from initial position

total_time_default = []
travelled_dist_default = []
straight_dist_default = []
states_default = {s:0 for s in possible_states}
for rdict, _, _ in results_default.values():
    elapsed_times = list(rdict.keys())
    prev_state = None
    prev_time = 0
    for i,ti in enumerate(elapsed_times[1:]):
        if prev_state is None:
            prev_state = rdict[ti]['state']
            prev_time = int(ti)
        if prev_state != rdict[ti]['state']:
            states_default[prev_state] = states_default[prev_state] + (int(ti)-prev_time)/1000
            prev_time = int(ti)
        prev_state = rdict[ti]['state']

        if rdict[ti]['position'][2] <= landed_altitude:
            total_time_default.append(int(ti)/1000) # total time in seconds
            positions = []
            for tmp_ti in elapsed_times[1:][:i+1]:
                positions.append(rdict[tmp_ti]['position'])
            positions = np.asarray(positions)
            total_dist_line_int = np.sqrt(((positions[1:]-positions[:-1])**2).sum(axis=1)).sum()
            travelled_dist_default.append(total_dist_line_int) # total travelled distance

            straight_dist_default.append(np.sqrt(((positions[-1][:2]-positions[0][:2])**2).sum()))
            break

total_time_default = np.asarray(total_time_default)
travelled_dist_default = np.asarray(travelled_dist_default)
straight_dist_default = np.asarray(straight_dist_default)

In [ ]:
states_default

In [ ]:
total_time_default.mean(), travelled_dist_default.mean(), straight_dist_default.mean()

In [ ]:
total_time_default.sum()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(total_time_default, '.-')
plt.title("CLIP's default prompt engineering")
plt.ylabel("Total Time")
plt.xlabel("Experiment Number")
plt.xticks(ticks=range(len(total_time_default)), labels=[k[-3:] for k in results_default.keys()], rotation=90);

### Calculating metrics for the PEACE system

In [ ]:
# total time until reaching landing altitude
# total travelled distance until reaching landing altitude
# horizontal distance from initial position

total_time_peace = []
travelled_dist_peace = []
straight_dist_peace = []
states_peace = {s:0 for s in possible_states}
for rdict, _, _ in results_peace.values():
    elapsed_times = list(rdict.keys())
    prev_state = None
    prev_time = 0
    for i,ti in enumerate(elapsed_times[1:]):
        if prev_state is None:
            prev_state = rdict[ti]['state']
            prev_time = int(ti)
        if prev_state != rdict[ti]['state']:
            states_peace[prev_state] = states_peace[prev_state] + (int(ti)-prev_time)/1000
            prev_time = int(ti)
        prev_state = rdict[ti]['state']

        if rdict[ti]['position'][2] <= landed_altitude:
            total_time_peace.append(int(ti)/1000) # total time in seconds
            positions = []
            for tmp_ti in elapsed_times[1:][:i+1]:
                positions.append(rdict[tmp_ti]['position'])
            positions = np.asarray(positions)
            total_dist_line_int = np.sqrt(((positions[1:]-positions[:-1])**2).sum(axis=1)).sum()
            travelled_dist_peace.append(total_dist_line_int) # total travelled distance

            straight_dist_peace.append(np.sqrt(((positions[-1][:2]-positions[0][:2])**2).sum()))
            break

total_time_peace = np.asarray(total_time_peace)
travelled_dist_peace = np.asarray(travelled_dist_peace)
straight_dist_peace = np.asarray(straight_dist_peace)

In [ ]:
states_peace

In [ ]:
total_time_peace.mean(), travelled_dist_peace.mean(), straight_dist_peace.mean()

In [ ]:
total_time_peace.sum()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(total_time_peace, '.-')
plt.title("PEACE")
plt.ylabel("Total Time")
plt.xlabel("Experiment Number")
plt.xticks(ticks=range(len(total_time_peace)), labels=[k[-3:] for k in results_peace.keys()], rotation=90);

### Calculating metrics for the original system

In [ ]:
# total time until reaching landing altitude
# total travelled distance until reaching landing altitude
# horizontal distance from initial position

total_time_original = []
travelled_dist_original = []
straight_dist_original = []
states_original = {s:0 for s in possible_states}
for rdict, _, _ in results_original.values():
    elapsed_times = list(rdict.keys())
    prev_state = None
    prev_time = 0
    for i,ti in enumerate(elapsed_times[1:]):
        if prev_state is None:
            prev_state = rdict[ti]['state']
            prev_time = int(ti)
        if prev_state != rdict[ti]['state']:
            states_original[prev_state] = states_original[prev_state] + (int(ti)-prev_time)/1000
            prev_time = int(ti)
        prev_state = rdict[ti]['state']
        if rdict[ti]['position'][2] <= landed_altitude:
            total_time_original.append(int(ti)/1000) # total time in seconds
            positions = []
            for tmp_ti in elapsed_times[1:][:i+1]:
                positions.append(rdict[tmp_ti]['position'])
            positions = np.asarray(positions)
            total_dist_line_int = np.sqrt(((positions[1:]-positions[:-1])**2).sum(axis=1)).sum()
            travelled_dist_original.append(total_dist_line_int) # total travelled distance

            straight_dist_original.append(np.sqrt(((positions[-1][:2]-positions[0][:2])**2).sum()))
            break

total_time_original = np.asarray(total_time_original)
travelled_dist_original = np.asarray(travelled_dist_original)
straight_dist_original = np.asarray(straight_dist_original)

In [ ]:
states_original

In [ ]:
total_time_original.mean(), travelled_dist_original.mean(), straight_dist_original.mean()

In [ ]:
total_time_original.sum()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(total_time_original, '.-')
plt.title("DOVESEI")
plt.ylabel("Total Time")
plt.xlabel("Experiment Number")
plt.xticks(ticks=range(len(total_time_original)), labels=[k[-3:] for k in results_original.keys()], rotation=90);

In [ ]:
plt.xticks?

## Special: Figures



In [ ]:
with open("results_default.json", "r") as file:
    results_default = json.load(file)

with open("results_peace.json", "r") as file:
    results_peace = json.load(file)

with open("results_original.json", "r") as file:
    results_original = json.load(file)

In [ ]:
experiment_name = 'experiment_007'

experiment_dict = results_peace[experiment_name][0]
elapsed_times = list(experiment_dict.keys())
positions_peace = []
for ti in elapsed_times[1:]:
    positions_peace.append(experiment_dict[ti]['position'][:2])
positions_peace = np.asarray(positions_peace)


experiment_dict = results_default[experiment_name][0]
elapsed_times = list(experiment_dict.keys())
positions_default = []
for ti in elapsed_times[1:]:
    positions_default.append(experiment_dict[ti]['position'][:2])
positions_default = np.asarray(positions_default)

experiment_dict = results_original[experiment_name][0]
elapsed_times = list(experiment_dict.keys())
positions_focus = []
for ti in elapsed_times[1:]:
    positions_focus.append(experiment_dict[ti]['position'][:2])
positions_focus = np.asarray(positions_focus)

plt.plot(positions_peace[:,0], positions_peace[:,1],
         linestyle='--', linewidth=4, color='blue', label='DOVESEI-PEACE')
plt.plot([positions_peace[-1,0]], [positions_peace[-1,1]],
         marker='*', markersize=20, color='blue')

plt.plot(positions_default[:,0], positions_default[:,1],
         linestyle='-.', linewidth=2, color='green', label='DOVESEI-DEF')
plt.plot([positions_default[-1,0]], [positions_default[-1,1]],
         marker='*', markersize=20, color='green')

plt.plot(positions_focus[:,0], positions_focus[:,1],
         linestyle='-.', linewidth=2, color='red', label='DOVESEI')
plt.plot([positions_focus[-1,0]], [positions_focus[-1,1]],
         marker='*', markersize=20, color='red')

plt.plot([positions_focus[1,0]], [positions_focus[1,1]],
         marker='.', markersize=20, color='black', label='Start')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title(f"Experiment {experiment_name[-3:]}")
plt.legend()
plt.show()

### Videos

In [ ]:
# (sub)folder where the experiments were saved
main_folder = "PEACE"

experiment_folders = sorted(glob(main_folder + '/*/'))
for folder in experiment_folders:
    if experiment_name in folder:
        heatmap_codes = np.asarray([int(hm.split('/')[-1].split('_')[-1].split('.')[0]) for hm in sorted(glob(folder + '/heatmap*'))])
        main_filename = folder + experiment_name
        with open(main_filename + ".json", "r") as file:
            experiment_dict = json.load(file)
        with open(main_filename + ".txt", "r") as file:
            init_lat, init_lon = [float(v.split('=')[1]) for v in file.readline().split(',')]
        elapsed_times = list(experiment_dict.keys())

        positions = []
        success = False
        for ti in elapsed_times[1:]:
            closest_ti = heatmap_codes[np.argsort(abs(heatmap_codes-int(ti)))[0]]
            heatmap = np.asarray(Image.open(f"{folder}heatmap_{closest_ti:07}.png").convert('RGB').resize((324,324)))
            positions.append(experiment_dict[ti]['position'][:2])
            diffPos = experiment_dict[ti]['position']
            bearing = math.degrees(math.atan2(-diffPos[1],diffPos[0]))+90
            distance = np.linalg.norm(diffPos[:2]) # relative distance
            final_lat, final_lon = avg.getPointAtDistance(init_lat, init_lon, distance, bearing)
            img = np.asarray(avg.getAerialImage(final_lat, final_lon, 0, diffPos[2], FOV, (324,324)))
            square_size = avg.squareAtCamera(diffPos[2],FOV)
            pixel_per_meter = 324/square_size
            fig, axs = plt.subplots(1,1, figsize=(15,10))
            axs.imshow(img)
            axs.imshow(heatmap, alpha=0.3)
            axs.scatter([324/2],[324/2],500,marker='+',color='r',linewidths=3)
            plt.text(10, 20, f"State: {experiment_dict[ti]['state']}", dict(size=25, color='white'))
            plt.text(10, 40, f"Approx. Altitude: {diffPos[2]:.0f}m", dict(size=25, color='white'))
            axs.axis('off')
            plt.tight_layout()
            plt.savefig(f"video/{experiment_name}_{int(ti):07}.png")
            plt.close() # will close the plot
            if experiment_dict[ti]['position'][2] <= landed_altitude:
                success = True
                break


In [ ]:
experiment_dict = results_peace[experiment_name][0]
elapsed_times = list(experiment_dict.keys())
positions_peace = []
for ti in elapsed_times[1:]:
    positions_peace.append(experiment_dict[ti]['position'][:2])
positions_peace = np.asarray(positions_peace)


plt.plot(positions_peace[:,0], positions_peace[:,1],
         linestyle='--', linewidth=4, color='blue', label='DOVESEI-PEACE')
plt.plot([positions_peace[-1,0]], [positions_peace[-1,1]],
         marker='*', markersize=20, color='blue')


plt.plot([positions_peace[1,0]], [positions_peace[1,1]],
         marker='.', markersize=20, color='black', label='Start')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title(f"Experiment {experiment_name[-3:]}")
plt.legend()

# plt.savefig("test.png")
# plt.close() # will close the plot

In [ ]:
fig, ax = plt.subplots()

# Save the chart so we can loop through the bars below.
bars = ax.bar(
    x=[0,1],
    height=[len(results_original),len(results_peace)],
    tick_label=['DOVESEI', 'PEACE']
)

# Axis formatting.
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#DDDDDD')
ax.tick_params(bottom=False, left=False)
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)

secax = ax.secondary_yaxis('right')
secax.set_ylabel('Average Path Length [m]')
secax.spines['top'].set_visible(False)
secax.spines['right'].set_visible(False)
secax.spines['left'].set_visible(False)
secax.spines['top'].set_visible(False)
secax.spines['bottom'].set_color('#DDDDDD')

# Add text annotations to the top of the bars.
bar_color = bars[0].get_facecolor()
for bar in bars:
  ax.text(
      bar.get_x() + bar.get_width() / 2,
      bar.get_height() + 0.3,
      round(bar.get_height(), 1),
      horizontalalignment='center',
      color=bar_color,
      weight='bold'
  )

# Add labels and a title. Note the use of `labelpad` and `pad` to add some
# extra space between the text and the tick labels.
ax.set_ylabel('Successfully reached a safe landing spot', labelpad=15, color='#333333')
ax.set_ylim(0,50)

fig.tight_layout()

In [ ]:
results_peace['experiment_006'][0][list(experiment_dict.keys())[1]]